# 0. Prepare Env and Data

In [1]:
using CSV, DataFrames, JuMP, Gurobi, Ipopt, LinearAlgebra, Plots, Random

In [2]:
const GRB_ENV = Gurobi.Env()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-03


Gurobi.Env(Ptr{Nothing} @0x00000000d7f8c1f0, false, 0)

In [3]:
# Risk = CSV.read("Risk_cov - SDP.csv", DataFrame,header=0);
Risk = CSV.read("Risk_cov.csv", DataFrame, header=0);
Return = CSV.read("Return_mean.csv", DataFrame, header=0);

Risk = Matrix(Risk);
Return_mean = Matrix(Return)[:, 1];
Return_lower = Matrix(Return)[:, 2];
Return_upper = Matrix(Return)[:, 3];

In [4]:
Return

,Column1,Column2,Column3
,Float64,Float64,Float64
1,-0.69,-0.7,-0.66
2,-0.671,-0.7,-0.66
3,-0.671,-0.7,-0.66
4,-0.671,-0.7,-0.66
5,-0.671,-0.7,-0.66
6,0.247,0.23,0.28
7,0.252,0.23,0.28
8,-0.671,-0.7,-0.66
9,-0.671,-0.7,-0.66


### test

In [5]:
println(sum(Return_mean' * Risk * Return_mean))
println(sum(Return_mean .* Return_mean))

30572.266417309205
7.7458078100000005


In [6]:
b = [0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3]
println(sum(b .* Return_mean))
println(sum(b' * Risk * b))
print(typeof(b))

-17.274
6005.8084
Vector{Int64}

### Experiments

In [21]:
# real
return_real_org = CSV.read("Return_real.csv", DataFrame)
return_real = Matrix(return_real_org[:, 2:end])

# benchmark
return_benchmark_org = CSV.read("Return_benchmark.csv", DataFrame)
return_benchmark = Matrix(return_benchmark_org[:, 2:end])

# forecast
return_mean_org = CSV.read("Return_mean_RF.csv", DataFrame)
return_mean = Matrix(return_mean_org[:, 2:end])
return_ub_org = CSV.read("Return_UB_RF.csv", DataFrame, header=false)
# return_ub = Matrix(return_ub_org[:, 2:end])
return_ub = Matrix(return_ub_org)
return_lb_org = CSV.read("Return_LB_RF.csv", DataFrame, header=false)
# return_lb = Matrix(return_lb_org[:, 2:end])
return_lb = Matrix(return_lb_org)

return_lstm_org = CSV.read("Return_mean_LSTM.csv", DataFrame, header=0)
return_lstm = Matrix(return_lstm_org)

# forecast standard deviation
std_org = CSV.read("Return_std.csv", DataFrame)
std_ = Matrix(std_org[:, 2:end])'
;

In [22]:
@show size(return_mean)
@show size(return_ub)
@show size(return_lstm)
@show size(std_)
;

size(return_mean) = (20, 30)
size(return_ub) = (20, 30)
size(return_lstm) = (20, 30)
size(std_) = (20, 30)


In [12]:
# return_mean .<= return_ub

In [13]:
# return_mean .>= return_lb

# 1. Deterministic Optimization

## 1.1 model_1: Risk as constraint, Return as objective

In [6]:
function model_1(Risk::Matrix, Return::Matrix; Risk_Constraint::Float64)
    n, n1 = size(Risk);
    n2, _ = size(Return);
    @assert n1 == n;
    @assert n2 == n;
    
    # Create the model
    model = Model(() -> Gurobi.Optimizer(GRB_ENV));

    # Decision Variables
    @variable(model, x[1:n] >= 0);

    # Objective Function
    @objective(model, Max, sum(Return[i] * x[i] for i=1:n));

    # Constraints
    # Risk
    @constraint(model, risk, sum(x[i] * Risk[i,j] * x[j] for i=1:n, j=1:n) <= Risk_Constraint);
    
    return model, x;
end

model_1 (generic function with 1 method)

In [27]:
buildtime_1 = @elapsed model1, x = model_1(Risk, Return_mean, Risk_Constraint=100.00);
solvetime_1 = @elapsed optimize!(model1)
@show buildtime_1
@show solvetime_1

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 0 rows, 20 columns and 0 nonzeros
Model fingerprint: 0xdb82ff1b
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  QMatrix range    [3e-02, 3e+04]
  Objective range  [3e-02, 7e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
  QRHS range       [1e+02, 1e+02]

User-callback calls 14, time in user-callback 0.00 sec


LoadError: Gurobi Error 10020: Constraint Q not PSD (diagonal adjustment of 4.9e+01 would be required). Set NonConvex parameter to 2 to solve model.

## 1.2 model_2: Return as constraint, Risk as objective

Indices:

$$
\begin{align}
i \in I \quad & \text{Asset index} \\
n = 20 \quad & \text{Number of company considered}
\end{align}
$$

Parameters/data:

$$
\begin{align}
\sum \quad & \text{Covariance matrix of assets} \\
\bar{r} \quad & \text{Expected return by forecast models} \\
R \quad & \text{Expected return lower limit} \\
r^l \quad & \text{Lower bound of expected return with a $95\%$ forecast confidence level} \\
r^u \quad & \text{Upper bound of expected return with a $95\%$ forecast confidence level} \\
\sigma \quad & \text{Forecast standard deviation of expected return with a given forecast confidence level} \\
U_1=\left\{u\in R^I, \ r_i^l \leq \bar{r_i} + u_i \leq r_i^u \right\} \quad & \text{Uncertainty set of element-wise expected return} \\
U_2=\left\{u\in R^I, \ -\Gamma \sqrt{n} \leq \sum_{i\in I} \frac{u_i}{\sigma_i} \leq \Gamma \sqrt{n} \right\} \quad & \text{Uncertainty set with center limit theorem (use $\Gamma=2$)} \\
\end{align}
$$

Decision variables:

$$
x \quad \text{Portfolio investment quantity}
$$

Formulation (deterministic):

$$
\begin{align}
\min \quad & x^T \sum x \\
\text{s.t.} \quad & \bar{r}^T x \geq R \\
& x_i \geq 0, \ \forall i \in I \\
\end{align}
$$

Formulation (robust):

$$
\begin{align}
\min \quad & x^T \sum x \\
\text{s.t.} \quad & \left(\bar{r} + u\right)^T x \geq R, \ u \in U \\
& x_i \geq 0, \ \forall i \in I \\
\end{align}
$$

Reformulation of robust optimization with uncertainty set $U_1$:

$$
\begin{align}
\min \quad & x^T \sum x \\
\text{s.t.} \quad & \bar{r}^T x - p^T \left(r^u - \bar{r} \right) - q^T \left(r^l - \bar{r} \right) \geq R \\
& p + q = x \\
& x_i, p_i \geq 0, \ q_i \leq 0, \ \forall i \in I \\
\end{align}
$$

Reformulation of robust optimization with uncertainty set $U_2$:

$$
\begin{align}
\min \quad & x^T \sum x \\
\text{s.t.} \quad & \bar{r}^T x - \Gamma \sqrt{n} p + \Gamma \sqrt{n} q \geq R \\
& \left(p + q\right) \sigma_i^{-1} = x_i, \ \forall i \in I \\
& x_i, p \geq 0, \ q \leq 0, \ \forall i \in I \\
\end{align}
$$

In [71]:
function model_2(Risk::Matrix, Return::Vector; Return_Constraint::Float64, solver::String="ipopt")
#     n, n1 = size(Risk);
#     n2, _ = size(Return);
#     @assert n1 == n;
#     @assert n2 == n;
    
    # n<=18, solverable with convex opt
    n = 20
    Risk = Risk[begin:n, begin:n]
    Return = Return[begin:n]
    
    # Create the model
    if solver == "gurobi"
        model = Model(() -> Gurobi.Optimizer(GRB_ENV))
        set_optimizer_attributes(model, "NonConvex" => 2, "TimeLimit" => 300);
    else
        model = Model(Ipopt.Optimizer)
        set_optimizer_attributes(model, "max_cpu_time" => 60.0, "print_level" => 0)
    end

    # Decision Variables
    @variable(model, x[1:n] >= 0);

    # Objective Function
    @objective(model, Min, x' * Risk * x);

    # Constraints
    # Return
    @constraint(model, sum(Return .* x) >= Return_Constraint);
    
    return model, x;
end

model_2 (generic function with 2 methods)

In [72]:
buildtime_2 = @elapsed model2, x = model_2(Risk, Return_mean, Return_Constraint=0.20);
solvetime_2 = @elapsed optimize!(model2)
@show buildtime_2
@show solvetime_2

buildtime_2 = 0.0752892
solvetime_2 = 0.0033427


0.0033427

In [204]:
res_benchmark = zeros(size(return_benchmark)[1], size(return_benchmark)[2])
res_obj = zeros(size(return_benchmark)[2])

for i in 1:size(return_benchmark)[2]
    @show i
    buildtime_2 = @elapsed model2, x = model_2(Risk, return_benchmark[:, i], Return_Constraint=0.05);
    solvetime_2 = @elapsed optimize!(model2)
    @show buildtime_2
    @show solvetime_2
    res_benchmark[:, i] = value.(x)
    res_obj[i] = objective_value(model2)
    @show objective_value(model2)
end

i = 1
buildtime_2 = 0.0003501
solvetime_2 = 0.004293
objective_value(model2) = 317.7727685791542
i = 2
buildtime_2 = 0.000234
solvetime_2 = 0.0046483
objective_value(model2) = 140.44167826665222
i = 3
buildtime_2 = 0.0001961
solvetime_2 = 0.0036603
objective_value(model2) = 189.79653248719174
i = 4
buildtime_2 = 0.0001837
solvetime_2 = 0.0038578
objective_value(model2) = 102.71253172907127
i = 5
buildtime_2 = 0.000182
solvetime_2 = 0.0039969
objective_value(model2) = 111.64060980092493
i = 6
buildtime_2 = 0.0002365
solvetime_2 = 0.0038958
objective_value(model2) = 130.26616781723664
i = 7
buildtime_2 = 0.0001968
solvetime_2 = 0.0040355
objective_value(model2) = 206.25843309338197
i = 8
buildtime_2 = 0.000193
solvetime_2 = 0.0038324
objective_value(model2) = 227.54036405702
i = 9
buildtime_2 = 0.0002303
solvetime_2 = 0.0038128
objective_value(model2) = 138.45002892900175
i = 10
buildtime_2 = 0.0002174
solvetime_2 = 0.0044485
objective_value(model2) = 138.60505636082806
i = 11
buildtime_

In [208]:
sum(res_benchmark .* return_real, dims=1)
@show sum(sum(res_benchmark .* return_real, dims=1) .<= 0.) / 30
@show sum(res_obj)/30;

sum(sum(res_benchmark .* return_real, dims = 1) .<= 0.0) / 30 = 0.5333333333333333
sum(res_obj) / 30 = 118.00858044886667


118.00858044886667

In [190]:
@show sum(res_benchmark .* return_real, dims=1)

sum(res_benchmark .* return_real, dims = 1) = [-0.3083774181679557 -0.2015440648828243 0.457453607339615 0.15852597045468278 0.08429897774809443 -0.09195669502375596 -0.11448510324630978 -0.04970740857086409 0.025245277126145927 -0.28218921633551336 -0.7619945769390999 -0.1519340634328124 0.2249243014728039 -0.21210090896465034 -0.18017151344678886 0.20792888782981736 -0.16892215978149108 -0.36442962628810144 0.8077640662651554 0.6167267005941021 0.8128134037621096 -0.2858777725687617 -0.24754161989565604 2.4614538048625363 1.6759563752220994 0.16840008384223912 -0.17442214704692766 -0.12675298422878445 0.7225213762479863 0.5284866542366428]


1×30 Matrix{Float64}:
 -0.308377  -0.201544  0.457454  0.158526  …  -0.126753  0.722521  0.528487

In [209]:
res_lstm = zeros(size(return_lstm)[1], size(return_lstm)[2])
res_obj = zeros(size(return_lstm)[2])

for i in 1:size(return_lstm)[2]
    @show i
    buildtime_2 = @elapsed model2, x = model_2(Risk, return_lstm[:, i], Return_Constraint=0.05);
    solvetime_2 = @elapsed optimize!(model2)
    @show buildtime_2
    @show solvetime_2
    res_lstm[:, i] = value.(x)
    res_obj[i] = objective_value(model2)
    @show objective_value(model2)
end

i = 1
buildtime_2 = 0.0003309
solvetime_2 = 0.0046989
objective_value(model2) = 44.73779747109608
i = 2
buildtime_2 = 0.0002533
solvetime_2 = 0.0035783
objective_value(model2) = 67.30525804372579
i = 3
buildtime_2 = 0.0003113
solvetime_2 = 0.0033176
objective_value(model2) = 3.0670874103334205
i = 4
buildtime_2 = 0.0001898
solvetime_2 = 0.0046779
objective_value(model2) = 0.02727092671733282
i = 5
buildtime_2 = 0.000193
solvetime_2 = 0.0045788
objective_value(model2) = 1.672200821411531
i = 6
buildtime_2 = 0.0002042
solvetime_2 = 0.003441
objective_value(model2) = 6.3651859468952745
i = 7
buildtime_2 = 0.000203
solvetime_2 = 0.0042964
objective_value(model2) = 1.8671434089532466
i = 8
buildtime_2 = 0.0001856
solvetime_2 = 0.0041205
objective_value(model2) = 2.5702912070135184
i = 9
buildtime_2 = 0.0001838
solvetime_2 = 0.0040671
objective_value(model2) = 0.12631799201919214
i = 10
buildtime_2 = 0.0002343
solvetime_2 = 0.0037253
objective_value(model2) = 6.676603730147235
i = 11
buildti

In [210]:
sum(res_lstm .* return_real, dims=1)
@show sum(sum(res_lstm .* return_real, dims=1) .<= 0.) / 30
@show sum(res_obj)/30

sum(sum(res_lstm .* return_real, dims = 1) .<= 0.0) / 30 = 0.43333333333333335
sum(res_obj) / 30 = 129.97218397512708


129.97218397512708

In [203]:
@show sum(res_lstm .* return_real, dims=1)

sum(res_lstm .* return_real, dims = 1) = [-0.2029270220126376 0.1554757464039938 0.09145636466281266 -0.02771456731495299 0.011459786255982873 -0.012909799344847182 -0.026290090017077626 0.21789311964857583 -0.043198243239499445 -0.028863018717023247 -0.0550097730970638 0.07092275707335066 -0.009746229788377515 0.007983720629232834 4.992075149552564e-10 -0.015759381734867225 -0.016351617265719452 4.912476620647045e-9 0.36201532548121595 0.05133105843684886 0.08682841138574746 -0.020355789387010624 -0.08441928938698591 0.32118619919835484 0.1159518752299802 0.01911598132841714 -0.042967287712559206 0.01851880777641786 0.28353712158764527 0.07719710056997546]


1×30 Matrix{Float64}:
 -0.202927  0.155476  0.0914564  …  0.0185188  0.283537  0.0771971

In [39]:
function model_robust(Risk::Matrix, Return::Vector, LB::Vector, UB::Vector; Return_Constraint::Float64, solver::String="ipopt")
#     n, n1 = size(Risk);
#     n2, _ = size(Return);
#     @assert n1 == n;
#     @assert n2 == n;
    
    # n<=18, solverable with convex opt
    n = 20
    Risk = Risk[begin:n, begin:n]
    mean = Return[begin:n]
    upper = UB[begin:n]
    lower = LB[begin:n]
    
    # Create the model
    if solver == "gurobi"
        model = Model(() -> Gurobi.Optimizer(GRB_ENV))
        set_optimizer_attributes(model, "NonConvex" => 2, "TimeLimit" => 300);
    else
        model = Model(Ipopt.Optimizer)
        set_optimizer_attributes(model, "max_cpu_time"=> 60.0, "print_level" => 0)
    end
    
    # Decision Variables
    @variable(model, x[1:n] >= 0)
    @variable(model, p[1:n] >= 0)
    @variable(model, q[1:n] <= 0)

    # Objective Function
    @objective(model, Min, x' * Risk * x);

    # Constraints
    # Return
    # @constraint(model, sum(Return .* x) >= Return_Constraint)
    @constraint(model, sum(mean .* x) - sum(p .* (upper .- mean)) - sum(q .* (lower .- mean)) >= Return_Constraint)
    @constraint(model, [i=1:n], p[i] + q[i] == x[i])
    
    return model, x;
end

model_robust (generic function with 1 method)

In [40]:
buildtime_r = @elapsed model_r, x = model_robust(Risk, Return_mean, Return_lower, Return_upper, Return_Constraint=0.20, solver="ipopt");
solvetime_r = @elapsed optimize!(model_r)
@show buildtime_r
@show solvetime_r

buildtime_r = 0.4294583
solvetime_r = 0.010465


0.010465

In [41]:
@show opt_x = value.(x)
@show sum(opt_x .* Return_mean, dims=1)

opt_x = value.(x) = [-9.941962468135873e-9, -9.356562758714078e-9, -9.8269965608822e-9, -9.834972606373807e-9, 0.004117853441897887, 0.2532463523925272, 0.6634533282233637, -9.647599534303717e-9, -9.650809357014615e-9, -9.715546820676875e-9, -9.647763318997856e-9, -9.672857960934387e-9, -7.418384644931496e-9, -9.71232451797163e-9, -9.611970913425577e-9, -9.645398998903712e-9, -9.720430070641989e-9, -9.65598582278162e-9, -9.682855430587847e-9, -9.567017239964322e-9]
sum(opt_x .* Return_mean, dims = 1) = [0.2269791120016006]


1-element Vector{Float64}:
 0.2269791120016006

In [42]:
res_mean = zeros(size(return_mean)[1], size(return_mean)[2])
res_obj = zeros(size(return_mean)[2])

for i in 1:size(return_benchmark)[2]
    @show i
    buildtime_r = @elapsed model_r, x = model_robust(Risk, return_mean[:, i], return_lb[:, i], return_ub[:, i], Return_Constraint=0.05);
    solvetime_r = @elapsed optimize!(model_r)
    @show buildtime_r
    @show solvetime_r
    res_mean[:, i] = value.(x)
    res_obj[i] = objective_value(model_r)
    @show objective_value(model_r)
end

i = 1
buildtime_r = 0.0088081
solvetime_r = 0.0223572
objective_value(model_r) = 1.1344451207989445e7
i = 2
buildtime_r = 0.0006491
solvetime_r = 0.0213188
objective_value(model_r) = 3.80489695330483e8
i = 3
buildtime_r = 0.0006045
solvetime_r = 0.0170878
objective_value(model_r) = 15.699456374670989
i = 4
buildtime_r = 0.0005656
solvetime_r = 0.0171602
objective_value(model_r) = 0.2999610601279826
i = 5
buildtime_r = 0.0005929
solvetime_r = 0.0171497
objective_value(model_r) = 27.686895117631636
i = 6
buildtime_r = 0.0004604
solvetime_r = 0.0148474
objective_value(model_r) = 452.11370570487577
i = 7
buildtime_r = 0.0005519
solvetime_r = 0.0304401
objective_value(model_r) = 2450.99327610822
i = 8
buildtime_r = 0.0005839
solvetime_r = 0.0228417
objective_value(model_r) = 96826.99853222547
i = 9
buildtime_r = 0.0004754
solvetime_r = 0.0135722
objective_value(model_r) = 2.5930591240645726
i = 10
buildtime_r = 0.0005781
solvetime_r = 0.021034
objective_value(model_r) = 1022.2975979047159
i

In [45]:
res_mean
@show sum(res_mean .* return_mean, dims=1)
@show sum(sum(res_mean .* return_real, dims=1) .<= 0.00) / 30
@show sum(res_obj)/30;

sum(res_mean .* return_mean, dims = 1) = [0.16015418514231392 0.7546014415479351 0.10456009896999974 0.0940513295260358 0.10663329506118682 0.20310225426394543 0.0936246978142851 0.07530891771944562 0.13708933697079032 0.17314561403498296 0.13528926079469697 0.10857931371440284 0.09447478468544186 0.12482413724445333 1.7635269768920512e-9 0.0841680726500781 3.3461971737723333e-10 2.440101563420648e-9 1.3010572688035946e-9 0.12157247348114193 0.07162233334001508 0.10917818412606502 0.07773197143629731 2.360890211407901e-9 0.0963012568796575 0.10033051675668035 0.07497994479584616 1.893872045036164e-11 1.1583623673914055e-11 5.380844050616696e-11]
sum(sum(res_mean .* return_real, dims = 1) .<= 0.0) / 30 = 0.4666666666666667
sum(res_obj) / 30 = 1.3064703062460888e7


In [191]:
@show sum(res_mean .* return_real, dims=1);

sum(res_mean .* return_real, dims = 1) = [0.005731641071250034 -1.344834250356231 0.21980931185973743 -0.09191416600271204 0.060436562209429265 -0.1981585621657243 -0.04501048595896376 0.17100220384758397 -0.19499465845613673 -0.10198276665166142 -0.216806388782776 0.17343525957510206 0.01691875135885914 0.021045538640016025 6.490229931944388e-11 -0.09197173513192113 1.2270308889909324e-9 3.6261561709996055e-9 -2.2828001876020793e-9 0.20399282135992255 0.15660483037791956 -0.03887485174229487 -0.065003131465921 -1.8594732229960958e-9 0.16337390050629497 0.061130297327723813 -0.07787830124809939 1.5558466103157457e-11 4.979840772450617e-11 -2.0897247739996814e-12]


In [32]:
function model_robust_clt(Risk::Matrix, Return::Vector, STD::Vector; Return_Constraint::Float64, solver::String="ipopt")
        
    # n<=18, solverable with convex opt
    n = 20
    
    Gamma = 2
    sqrt_n = sqrt(n)

    Risk = Risk[begin:n, begin:n]
    mean = Return[begin:n]
    std = STD[begin:n]
    
    # Create the model
    if solver == "gurobi"
        model = Model(() -> Gurobi.Optimizer(GRB_ENV))
        set_optimizer_attributes(model, "NonConvex" => 2, "TimeLimit" => 300);
    else
        model = Model(Ipopt.Optimizer)
        set_optimizer_attributes(model, "max_cpu_time"=> 60.0, "print_level" => 0)
    end
    
    # Decision Variables
    @variable(model, x[1:n] >= 0)
    @variable(model, p >= 0)
    @variable(model, q <= 0)

    # Objective Function
    @objective(model, Min, x' * Risk * x);

    # Constraints
    # Return
    # @constraint(model, sum(Return .* x) >= Return_Constraint)
    @constraint(model, sum(mean .* x) - Gamma * sqrt_n * p + Gamma * sqrt_n * q >= Return_Constraint)
    @constraint(model, [i=1:n], std[i]^(-1) * (p + q) == x[i])
    
    return model, x;
end

model_robust_clt (generic function with 1 method)

In [33]:
buildtime_r = @elapsed model_r, x = model_robust_clt(Risk, Return_mean, std_[:,1], Return_Constraint=0.20, solver="ipopt");
solvetime_r = @elapsed optimize!(model_r)
@show buildtime_r
@show solvetime_r

buildtime_r = 0.2840931
solvetime_r = 0.0178712


0.0178712

In [34]:
@show opt_x = value.(x)
@show sum(opt_x .* Return_mean, dims=1)

opt_x = value.(x) = [-6.7080614335797685e-9, -9.997916386208164e-9, -5.314929694229839e-9, -2.758442976725712e-9, -9.16652931379393e-9, -6.5068031299177e-9, -6.109125283124549e-9, -6.667247079528243e-9, -9.997916386208157e-9, -6.8755283496355145e-9, -9.999455736154791e-9, -2.772328219190158e-9, -3.999076858646436e-9, -6.0446918231861165e-9, -4.622587853297483e-9, -6.66724707952827e-9, -9.999424787979187e-9, -9.999456705854635e-9, -9.090826895487021e-9, -9.99945462252425e-9]
sum(opt_x .* Return_mean, dims = 1) = [8.18953580817091e-8]


1-element Vector{Float64}:
 8.18953580817091e-8

In [35]:
res_mean = zeros(size(return_mean)[1], size(return_mean)[2])
res_obj = zeros(size(return_mean)[2])

for i in 1:size(return_benchmark)[2]
    @show i
    buildtime_r = @elapsed model_r, x = model_robust_clt(Risk, return_mean[:, i], std_[:, i], Return_Constraint=0.05);
    solvetime_r = @elapsed optimize!(model_r)
    @show buildtime_r
    @show solvetime_r
    res_mean[:, i] = value.(x)
    res_obj[i] = objective_value(model_r)
    @show objective_value(model_r)
end

i = 1
buildtime_r = 0.00664
solvetime_r = 0.0173821
objective_value(model_r) = 4.824827808798204e-15
i = 2
buildtime_r = 0.0004072
solvetime_r = 0.0168527
objective_value(model_r) = 9.257183973088379e-18
i = 3
buildtime_r = 0.0006319
solvetime_r = 0.0230409
objective_value(model_r) = 3.2865629955518762e-15
i = 4
buildtime_r = 0.0005049
solvetime_r = 0.0207889
objective_value(model_r) = 2.5970349676729307e-15
i = 5
buildtime_r = 0.0003061
solvetime_r = 0.0100456
objective_value(model_r) = 521563.5050993641
i = 6
buildtime_r = 0.0004356
solvetime_r = 0.0187648
objective_value(model_r) = 8.666611300136899e-16
i = 7
buildtime_r = 0.0005354
solvetime_r = 0.0252135
objective_value(model_r) = 4.030313227464627e-16
i = 8
buildtime_r = 0.0006562
solvetime_r = 0.0291321
objective_value(model_r) = 1.6334815274316033e-17
i = 9
buildtime_r = 0.0005405
solvetime_r = 0.0125572
objective_value(model_r) = 104371.77499836888
i = 10
buildtime_r = 0.0005249
solvetime_r = 0.0187948
objective_value(model_r)

In [47]:
res_mean
@show sum(res_mean .* return_mean, dims=1)
@show sum(sum(res_mean .* return_real, dims=1) .<= 0.00) / 30
@show sum(res_obj)/30;

sum(res_mean .* return_mean, dims = 1) = [0.16015418514231392 0.7546014415479351 0.10456009896999974 0.0940513295260358 0.10663329506118682 0.20310225426394543 0.0936246978142851 0.07530891771944562 0.13708933697079032 0.17314561403498296 0.13528926079469697 0.10857931371440284 0.09447478468544186 0.12482413724445333 1.7635269768920512e-9 0.0841680726500781 3.3461971737723333e-10 2.440101563420648e-9 1.3010572688035946e-9 0.12157247348114193 0.07162233334001508 0.10917818412606502 0.07773197143629731 2.360890211407901e-9 0.0963012568796575 0.10033051675668035 0.07497994479584616 1.893872045036164e-11 1.1583623673914055e-11 5.380844050616696e-11]
sum(sum(res_mean .* return_real, dims = 1) .<= 0.0) / 30 = 0.4666666666666667
sum(res_obj) / 30 = 1.3064703062460888e7


In [38]:
@show sum(res_mean .* return_real, dims=1);

sum(res_mean .* return_real, dims = 1) = [4.7824392222330734e-11 2.2847569123002677e-12 -2.5887757211170598e-11 3.0934430211771686e-11 0.08147948405383186 -1.0385100162651227e-13 1.9791264308184735e-12 3.3376360647391784e-12 0.016691707516735967 -8.721322525137574e-12 8.43984254534255e-13 -2.721489791760461e-13 2.5568556349739814e-12 -1.6378824240673323e-11 8.324960350797576e-11 -4.140963055897689e-14 1.3084066878804998e-12 2.0931054472944132e-9 -3.684540202891969e-12 -0.1555756715852859 -1.2941373329532825e-12 1.3297673219600455e-11 -1.3105175210062265e-11 -1.388128997677225e-9 1.1179930787683501e-11 7.367206530021907e-12 -1.1547399260820283e-10 3.6553872555263884e-13 4.573402623831599e-12 -1.533647359170263e-12]


# 2. Robust Optimization

## 2.1 RO_1: Return as constraint, Risk as objective